In [43]:
import pandas as pd
import os 
import json
from google.cloud import bigquery
from google.api_core.exceptions import NotFound
from datetime import datetime
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "credentials_gpe-analytics.json"

In [44]:
new_preds = pd.read_json('../data/predictions/final_preds/all_preds.json',orient='records',lines=True)

In [45]:
# Constants 
today = datetime.today().strftime('%Y%m%d')
gcs_folder = f'project-google-arxr-analytics-{today}'
gcs_dataset = gcs_folder.replace("-", "_")
gcs_bucket = 'mm-gpe-data'

# What



1. New twitter/news/insta needs combined with previous table 
CREATE new_updated_table
SELECT * FROM current_table
JOIN new_table 
WHERE current_table.updated_source=Reddit



In [46]:
all_preds = pd.read_json('../data/predictions/final_preds/all_preds.json',orient='records',lines=True)
new_preds = all_preds.loc[~all_preds['Page Type'].isna()].reset_index(drop=True) # Remove all Reddit values 

In [ ]:
#Optional
old_preds = pd.read_json('../data/predictions/final_preds/old-preds.json',orient='records',lines=True)
old_preds = old_preds.loc[old_preds['Page Type'].isna()].reset_index(drop=True)

In [47]:
#final_preds = pd.concat([new_preds, old_preds]).reset_index(drop=True)
final_preds = pd.concat([new_preds]).reset_index(drop=True)
final_preds.to_json(f"../data/predictions/final_preds/{today}_final_preds.json",orient="records",lines=True)

In [50]:
final_preds = pd.read_json(f'../data/predictions/final_preds/{today}_final_preds.json',orient='records',lines=True)

def all_preds_configuration(df):    
    # Change date column to timestamp
    if 'date' in df:
        df['date'] = df['date'].combine_first(df['Date'])
    else:
        df['date']  = df['Date']

    if 'author' in df:
        df['author'] = df['author'].combine_first(df['Author'])
    else:
        df['author'] = df['Author']

    # #Combine author for the news 
    df['author'] = df['author'].combine_first(df['Domain'])
        
    # # Now convert timestamp to string '2022-12-12'
    df['date'] = df['date'].dt.strftime('%Y-%m-%d')
    
    # # Twitter
    df['Engagements'] = df.apply(lambda x: x['Twitter Likes'] + x['Twitter Retweets'],axis=1)
    # # Reddit
    if 'redditScore' in df:
       df['engagements'] = df['redditScore'].combine_first(df['Engagements'])
    else:
       df['engagements'] = df['Engagements']
    
    # # Create updated source column
    df['updated_source'] = df.apply(lambda x: x['Page Type'] if x['Page Type'] else 'reddit',axis=1)
    
    #df['link_url'] = df.apply(lambda x: x.uid if x.updated_source!='reddit' else x.url,axis=1)
    df['link_url'] = df['uid']
    bad_colums = [
        "Reach (new)", 
        "Google AR/VR: Companies (Parent - products)",
        "[Chrome AI Snap] Browsers",
        "[Chrome AI Snap] ChatGPT vs Bard",
        "[Chrome AI Snap] Search Engines",
        "Engagements",
        "Date",
        "Author",
        "Google AR/VR: Categories",
        "Google AR/VR: Companies",
        "Google AR/VR: Companies (Parent - products)",
        "Google AR/VR: Events",
        "Google AR/VR: Glasses",
        "Google AR/VR: Headsets"]
    df.drop(columns=bad_colums, inplace=True)
    
    return df 

updated_preds = all_preds_configuration(final_preds)
cols = updated_preds.columns
cols.sort_values()
for x in cols:
    print(x)

Unnamed: 0
Query Id
Query Name
Title
text
Full Text
uid
Domain
Sentiment
Emotion
Page Type
Language
Country Code
Continent Code
Continent
Country
Region Code
Region
City Code
Account Type
Assignment
Avatar
Category Details
Checked
City
Display URLs
Expanded URLs
Facebook Author ID
Facebook Comments
Facebook Likes
Facebook Role
Facebook Shares
Facebook Subtype
Full Name
Gender
Hashtags
Impact
Impressions
Instagram Comments
Instagram Followers
Instagram Following
Instagram Likes
Instagram Posts
Interest
Last Assignment Date
Latitude
Location Name
Longitude
Media Filter
Media URLs
Mentioned Authors
Priority
Professions
Resource Id
Short URLs
Starred
Status
Subtype
Tags
Thread Author
Thread Created Date
Thread Entry Type
Thread Id
Thread URL
Total Monthly Visitors
Twitter Author ID
Twitter Channel Role
Twitter Followers
Twitter Following
Twitter Reply Count
Twitter Reply to
Twitter Retweet of
Twitter Retweets
Twitter Likes
Twitter Tweets
Twitter Verified
Reddit Score
Reddit Score Upvote Ra

In [51]:
# Create Table 
client = bigquery.Client()

In [52]:
# Define table name, in format dataset.table_name
dataset_id = f'{gcs_folder.replace("-","_")}'
table = f'{dataset_id}.ar_xr_predictions_updated'

# Check if the dataset exists
dataset_ref = client.dataset(dataset_id)
try:
    client.get_dataset(dataset_ref, retry=bigquery.DEFAULT_RETRY)
except Exception as e:
    dataset = client.create_dataset(dataset_id)
    print(f"Dataset '{dataset.dataset_id}' created successfully.")
else:
    print(f"Dataset '{dataset_id}' already exists.")

# Load data to BQ
job = client.load_table_from_dataframe(updated_preds, table)

Dataset 'project_google_arxr_analytics_20240619' created successfully.


# Do Clustering 

In [53]:
# Preprocessing keys 
{"text":"Would you prefer for Apple to adapt to the industry standard of RCS? #fyp #android #apple #iphone #samsung #google #ipager #phone #greenbubble #bluebubble #rcs #androidvsiphone #getthemessage","uid":"https:\/\/www.tiktok.com\/@redirect-to\/video\/7287566836936346912","source":"tiktok","date":"2024-01-04"}.keys()

dict_keys(['text', 'uid', 'source', 'date'])

In [54]:
# label map keys 
{"source": "Platform", "author": "NULL", "engagements": "NULL", "link": "Video_URL", "uid": "Video_URL", "date": "date", "queryName": "NULL", "query_title": "Video_Title", "subreddit": "NULL", "sentiment": "NULL"}.keys()

dict_keys(['source', 'author', 'engagements', 'link', 'uid', 'date', 'queryName', 'query_title', 'subreddit', 'sentiment'])

In [56]:
preprocessed_data = final_preds[['date','text','uid','source']]
preprocessed_data.to_json('preprocessed_data.json',orient='records',lines=True)
os.system(f'gsutil cp preprocessed_data.json gs://{gcs_bucket}/{gcs_folder}/preprocessed/')

Copying file://preprocessed_data.json [Content-Type=application/json]...
- [1 files][481.9 KiB/481.9 KiB]                                                
Operation completed over 1 objects/481.9 KiB.                                    


0

In [57]:
label_map = {'source':'source',
            'author':'author',
            'engagements':'NULL',
            'link':'NULL',
            'uid':'uid',
            'date':'date',
            'queryName':'NULL',
            'query_title':'NULL',
            'subreddit':'NULL',
            'sentiment':'NULL'}


with open('data_label_map.json','w') as f:
    json.dump(label_map,f)
    
os.system(f'gsutil cp data_label_map.json gs://{gcs_bucket}/{gcs_folder}/label_maps/')


Copying file://data_label_map.json [Content-Type=application/json]...
/ [1 files][  195.0 B/  195.0 B]                                                
Operation completed over 1 objects/195.0 B.                                      


0

In [58]:
final_preds.to_json('raw_data.json',orient='records',lines=True)
os.system(f'gsutil cp raw_data.json gs://{gcs_bucket}/{gcs_folder}/raw/')


Copying file://raw_data.json [Content-Type=application/json]...
\ [1 files][  5.9 MiB/  5.9 MiB]                                                
Operation completed over 1 objects/5.9 MiB.                                      


0

# Run analytics [pipeline](https://console.cloud.google.com/vertex-ai/pipelines/locations/us-central1/runs/pipeline-googlae-arxr-analytics-20240427-run-1?project=gpe-analytics) to generate additional tables

* redditnlp_output
* custom_nlp_tags
* clusters

# Then, combine tables to create report dashboard

In [ ]:

# Define table name, in format dataset.table_name
dataset_id = f'{gcs_folder.replace("-","_")}'
    
create_dashboard = """
CREATE TABLE `{dataset_id}.view_dashboard_final_update` as SELECT 
    * 
    FROM 
    (
        SELECT 
        DISTINCT LEFT(text, 300) AS text, 
        uid, 
        category_1, 
        probability_1, 
        category_2, 
        probability_2, 
        subreddit_category_1,
        subreddit_probability_1
        FROM `{dataset_id}.redditnlp_output`
    )  
    LEFT JOIN (
        SELECT 
        updated_source ,link_url,uid,date_,query, tag, llm_label, engagements_ 
        FROM `{dataset_id}.ar_xr_predictions_table_updated`) USING (uid)
    LEFT JOIN (
        SELECT 
        * 
        FROM `{dataset_id}.clusters`) USING (uid) 
    LEFT JOIN (
        SELECT 
        * 
        FROM `{dataset_id}.custom_nlp_tags`) USING (uid) 
"""

# Execute the SQL query
client.query(create_dashboard)